**Diagnosis and Treatment Plan**

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0. langchain-groq python-docx
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-groq python-docx

**LLm for ai agents**

In [3]:
import os
#from google.colab import userdata
from langchain_groq import ChatGroq
class GroqChatLLM(ChatGroq):
    def __init__(self, temperature=0, model_name="llama-3.1-70b-versatile", api_key=None):
        api_key = "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"

        if not api_key:
            raise ValueError("GROQ_API_KEY must be provided either as an argument or in the environment variables.")

        # Directly call the superclass (ChatGroq) initialization
        super().__init__(
            temperature=temperature,
            model_name=model_name,
            api_key=api_key
        )

In [4]:
llm=GroqChatLLM()

function for downlaod the docx file


In [6]:

from crewai import Agent, Task, Crew, Process
import os
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI
from docx import Document
from io import BytesIO
import base64

load_dotenv()

# LLM object and API Key
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = "633c9f4f684e57cb300edf5a39a465f85f800c13"


def generate_docx(result):
    doc = Document()
    doc.add_heading('Healthcare Diagnosis and Treatment Recommendations', 0)
    doc.add_paragraph(result)
    bio = BytesIO()
    doc.save(bio)
    bio.seek(0)
    return bio

def get_download_link(bio, filename):
    b64 = base64.b64encode(bio.read()).decode()
    return f'<a href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}" download="{filename}">Download Diagnosis and Treatment Plan</a>'


**Tools**



**Search Tool:**

**Purpose:**
 A tool designed to search the web for relevant information based on a query.
**Usage:**
 Given a query (e.g., medical conditions or treatments), the tool retrieves web results to help the agent gather necessary data for diagnosis or treatment.
**Scrape Tool:**

**Purpose:**
 A tool used to extract detailed content from a specified website.
**Usage:**
After retrieving a web page using the search tool, the scrape tool extracts specific information (like treatment details or diagnosis-related content) for further analysis.

In [7]:
# Initialize Tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

**Agents**

###**Medical Diagnostician:**

**Role:** To analyze patient symptoms and medical history to provide a preliminary diagnosis.
**Specialty:** This agent uses medical knowledge and web tools to identify potential health conditions based on reported symptoms.

**Actions**: It analyzes input data, searches for relevant medical information, and presents possible diagnoses.

###**Treatment Advisor:**

**Role:** To recommend appropriate treatment plans based on the diagnosis.
**Specialty:** This agent crafts detailed, step-by-step treatment recommendations, considering diagnosis, medical history, and current symptoms. It uses web-based information to offer accurate treatment plans.

**Actions:** The agent searches for treatment guidelines, scrapes content for detailed insights, and provides recommendations on medication, lifestyle changes, and follow-up care.

In [8]:
# Define Agents
diagnostician = Agent(
    role="Medical Diagnostician",
    goal="Analyze patient symptoms and medical history to provide a preliminary diagnosis.",
    backstory="Specializes in diagnosing medical conditions using advanced algorithms and medical knowledge.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool],
    llm=llm,
    reasoning="Chain of Thought (step-by-step reasoning) and ReAct prompting (dynamic updates based on new information)."
)

treatment_advisor = Agent(
    role="Treatment Advisor",
    goal=(
        "Recommend detailed and step-by-step treatment plans based on the diagnosis provided by the Medical Diagnostician. "
        "Use the provided search and scrape tools to gather relevant information from reliable sources."
    ),
    backstory=(
        "This agent specializes in creating personalized treatment plans. It uses advanced algorithms, best practices in medicine, "
        "and web-sourced information to offer precise recommendations, including medications, lifestyle changes, and follow-up care."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool],  # Ensure tools are defined correctly
    llm=llm,
    action_handlers={
        "Search the internet": search_tool.run,  # Ensure action handlers are linked to tool methods
        "Scrape content": scrape_tool.run  # Scrape content from web using scrape tool
    }
)

**Tasks**

### **Diagnose Task:**

**Purpose:** To analyze symptoms and medical history and generate a preliminary diagnosis with a list of possible conditions.
**Process: **The Diagnostician Agent uses the tools to search for and provide a preliminary diagnosis based on the patient’s input.

### **Treatment Task:**

**Purpose:** To generate a step-by-step treatment plan based on the diagnosis and patient’s history.
**Process:** The Treatment Advisor Agent uses web tools to gather information about appropriate treatments and recommends medications, lifestyle changes, and follow-up care, tailored to the patient's specific diagnosis and symptoms.

In [9]:
# Define Tasks
diagnose_task = Task(
    description=(
        "1. Analyze the patient's symptoms ({symptoms}) and medical history ({medical_history}).\n"
        "2. Provide a preliminary diagnosis with possible conditions based on the provided information.\n"
        "3. Limit the diagnosis to the most likely conditions."
    ),
    expected_output="A preliminary diagnosis with a list of possible conditions.",
    agent=diagnostician
)

treatment_task = Task(
    description=(
        "1. Search the web for diagnosis and treatments using the provided tools (search_tool, scrape_tool) based on the patient's symptoms and diagnosis.\n"
        "2. Analyze the patient's medical history ({medical_history}) and current symptoms ({symptoms}).\n"
        "3. Provide a step-by-step treatment plan, including medications, lifestyle changes, and follow-up care."
    ),
    expected_output="A personalized and detailed treatment plan based on diagnosis and medical history.",
    agent=treatment_advisor
)

**Crew**

In [10]:
# Create Crew
crew = Crew(
    agents=[diagnostician, treatment_advisor],
    tasks=[diagnose_task, treatment_task],
    verbose=2
)

**Input:**

**bold text**

In [12]:
symptoms="bodyache; cough"
medical_history="covid"

result = crew.kickoff(inputs={"symptoms": symptoms, "medical_history": medical_history})

docx_file = generate_docx(result)



 [DEBUG]: == Working Agent: Medical Diagnostician
 [INFO]: == Starting Task: 1. Analyze the patient's symptoms (bodyache; cough) and medical history (covid).
2. Provide a preliminary diagnosis with possible conditions based on the provided information.
3. Limit the diagnosis to the most likely conditions.


> Entering new CrewAgentExecutor chain...
Thought: To analyze the patient's symptoms and medical history, I need to gather more information about the possible conditions that could be causing these symptoms.

Action: Search the internet
Action Input: {"search_query": "bodyache and cough and covid"} 


Search results: Title: Body Aches: COVID-19 Symptoms or Other Causes? What to do | Ada
Link: https://ada.com/covid/covid-19-symptom-body-aches/
Snippet: Body aches are a frequent symptom of COVID, and are often felt with other symptoms such as fatigue, cough, and headaches.
---
Title: COVID-19, flu, RSV, cold or allergies? How to tell the difference
Link: https://www.osfhealthcare.org/

In [ ]:
from IPython.display import Markdown, display

# Example usage
display(Markdown(result))


generating a .docx file containing healthcare diagnosis and treatment recommendations, and then providing a download link for that file. Here's a quick explanation of what each function does:

**generate_docx(result):**

Creates a .**docx** file using the python-docx library **(Document())**.
Adds a heading "Healthcare Diagnosis and Treatment Recommendations" to the document.
Inserts the diagnosis and treatment details **(result)** into the document.
The document is saved into an in-memory byte stream **(BytesIO)** and returned.
**get_download_link(bio, filename):**

Converts the **BytesIO** stream into a base64-encoded string.
Returns an **HTML** anchor tag that allows the user to download the .docx file with the provided filename.

In [ ]:
from docx import Document
from io import BytesIO
import base64
from IPython.display import display, HTML

# Generate .docx file
docx_file = generate_docx(result)

# Create download link
filename = "Diagnosis_Treatment_Plan.docx"
download_link = get_download_link(docx_file, filename)

# Display the download link in a notebook
display(HTML(download_link))
